In [19]:
import os
import cv2
import shutil
import numpy as np
import matplotlib.pyplot as plt

In [7]:
def resize_image(image):
    
    # Resize the image to 256x256 pixels
    resized_image = cv2.resize(image, (256, 256))
    
    return resized_image

In [3]:
def normalize_image(image):
    # Normalize the image
    normalized_image = image / 255.0
    
    return normalized_image

# Resizing images!

## For cubism

In [14]:
# Define the paths
original_image_dir = 'data/cubism/originalImage/'
resized_image_dir = 'data/cubism/resizedImage/'

# Get the list of all files in the originalImage directory
all_files = os.listdir(original_image_dir)

# Loop through each file
for file_name in all_files:
    # Construct the full path of the original image
    original_image_path = os.path.join(original_image_dir, file_name)
    
    # Read the original image
    image = cv2.imread(original_image_path)
    
    # Resize the image using the predefined function
    resized_image = resize_image(image)
    
    # Construct the full path of the resized image
    resized_image_path = os.path.join(resized_image_dir, file_name)
    
    # Save the resized and normalized image
    cv2.imwrite(resized_image_path, resized_image)


## For Impressionism

In [15]:
# Define the paths
original_image_dir = 'data/impressionism/originalImage/'
resized_image_dir = 'data/impressionism/resizedImage/'

# Get the list of all files in the originalImage directory
all_files = os.listdir(original_image_dir)

# Loop through each file
for file_name in all_files:
    # Construct the full path of the original image
    original_image_path = os.path.join(original_image_dir, file_name)
    
    # Read the original image
    image = cv2.imread(original_image_path)
    
    # Resize the image using the predefined function
    resized_image = resize_image(image)
    
    # Construct the full path of the resized image
    resized_image_path = os.path.join(resized_image_dir, file_name)
    
    # Save the resized and normalized image
    cv2.imwrite(resized_image_path, resized_image)


## For Post Impressionism

In [16]:
# Define the paths
original_image_dir = 'data/post-impressionism/originalImage/'
resized_image_dir = 'data/post-impressionism/resizedImage/'

# Get the list of all files in the originalImage directory
all_files = os.listdir(original_image_dir)

# Loop through each file
for file_name in all_files:
    # Construct the full path of the original image
    original_image_path = os.path.join(original_image_dir, file_name)
    
    # Read the original image
    image = cv2.imread(original_image_path)

    # Resize the image using the predefined function
    resized_image = resize_image(image)
    
    # Construct the full path of the resized image
    resized_image_path = os.path.join(resized_image_dir, file_name)
    
    # Save the resized and normalized image
    cv2.imwrite(resized_image_path, resized_image)


# Spliting the data into train and val

## For cubism

In [20]:
allFiles = os.listdir('data/cubism/resizedImage/')
os.makedirs('data/cubism/train', exist_ok=True)
os.makedirs('data/cubism/val', exist_ok=True)

# Splitting percentage is training 80% and validation 20%
splittingPercentage = 0.8

# Calculate the number of files for each split
numberOfFiles = len(allFiles)
trainSplit = int(splittingPercentage * numberOfFiles)

# Move the files into the corresponding train and validation directories
for index, file in enumerate(allFiles):
    if index < trainSplit:
        shutil.move('data/cubism/resizedImage/' + file, 'data/cubism/train/')
    else:
        shutil.move('data/cubism/resizedImage/' + file, 'data/cubism/val/')

# For impressionism

In [21]:
allFiles = os.listdir('data/impressionism/resizedImage/')
os.makedirs('data/impressionism/train', exist_ok=True)
os.makedirs('data/impressionism/val', exist_ok=True)

# Splitting percentage is training 80% and validation 20%
splittingPercentage = 0.8

# Calculate the number of files for each split
numberOfFiles = len(allFiles)
trainSplit = int(splittingPercentage * numberOfFiles)

# Move the files into the corresponding train and validation directories
for index, file in enumerate(allFiles):
    if index < trainSplit:
        shutil.move('data/impressionism/resizedImage/' + file, 'data/impressionism/train/')
    else:
        shutil.move('data/impressionism/resizedImage/' + file, 'data/impressionism/val/')

# For post-impressionism

In [22]:
allFiles = os.listdir('data/post-impressionism/resizedImage/')
os.makedirs('data/post-impressionism/train', exist_ok=True)
os.makedirs('data/post-impressionism/val', exist_ok=True)

# Splitting percentage is training 80% and validation 20%
splittingPercentage = 0.8

# Calculate the number of files for each split
numberOfFiles = len(allFiles)
trainSplit = int(splittingPercentage * numberOfFiles)

# Move the files into the corresponding train and validation directories
for index, file in enumerate(allFiles):
    if index < trainSplit:
        shutil.move('data/post-impressionism/resizedImage/' + file, 'data/post-impressionism/train/')
    else:
        shutil.move('data/post-impressionism/resizedImage/' + file, 'data/post-impressionism/val/')

In [24]:
import tensorflow as tf
from tensorflow.keras import models, losses
from tensorflow.keras.applications import VGG19

class StyleTransferLoss(tf.keras.losses.Loss):
    def __init__(self, content_weight=1, style_weight=1e5):
        super(StyleTransferLoss, self).__init__()
        self.content_weight = content_weight
        self.style_weight = style_weight

        # Define VGG19 with features only
        vgg19 = VGG19(weights='imagenet', include_top=False)
        self.vgg = tf.keras.Model(inputs=vgg19.input, outputs=vgg19.get_layer('block5_conv4').output)
        self.vgg.trainable = False

        self.content_loss = losses.MeanSquaredError()
        self.style_loss = StyleLoss()

    def call(self, generated, target, style_reference):
        # Compute content loss
        content_loss = self.content_loss(generated, target) * self.content_weight

        # Compute style loss
        style_loss = self.style_loss(generated, style_reference) * self.style_weight

        total_loss = content_loss + style_loss
        return total_loss

class StyleLoss(tf.keras.layers.Layer):
    def __init__(self):
        super(StyleLoss, self).__init__()

    def gram_matrix(self, x):
        features = tf.keras.backend.batch_flatten(tf.keras.backend.permute_dimensions(x, (2, 0, 1)))
        gram = tf.matmul(features, tf.transpose(features))
        return gram

    def call(self, generated, style_reference):
        generated_gram = self.gram_matrix(generated)
        style_reference_gram = self.gram_matrix(style_reference)
        loss = tf.reduce_mean(tf.square(generated_gram - style_reference_gram))
        return loss


In [34]:
# Cubism

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, UpSampling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

# Generator model
def build_generator():
    input_shape = (256, 256, 3)  # Adjust the input shape based on your requirements

    model = Sequential([
        # Upsampling block 1
        UpSampling2D(size=(2, 2), input_shape=input_shape),
        Conv2D(128, kernel_size=3, padding='same'),
        BatchNormalization(),
        Activation('relu'),

        # Upsampling block 2
        UpSampling2D(size=(2, 2)),
        Conv2D(64, kernel_size=3, padding='same'),
        BatchNormalization(),
        Activation('relu'),

        # Output block
        Conv2D(3, kernel_size=3, padding='same', activation='tanh')
    ])

    return model

# Discriminator model
def build_discriminator():
    input_shape = (256, 256, 3)  # Adjust the input shape based on your requirements

    model = Sequential([
        # Convolutional block 1
        Conv2D(64, kernel_size=3, strides=2, padding='same', input_shape=input_shape),
        BatchNormalization(),
        Activation('relu'),

        # Convolutional block 2
        Conv2D(128, kernel_size=3, strides=2, padding='same'),
        BatchNormalization(),
        Activation('relu'),

        # Flatten and output
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    return model

# Define the StyleTransferLoss as provided in the previous steps
style_transfer_loss = StyleTransferLoss()

# Combine generator and discriminator for training the generator
def build_gan(generator, discriminator):
    discriminator.trainable = False

    # Ensure that the generator's output shape matches the input shape of the discriminator
    noise_input = Input(shape=generator.input_shape[1:])
    generated_image = generator(noise_input)
    validity = discriminator(generated_image)

    # Combine the generator and discriminator in a model
    model = Model(inputs=noise_input, outputs=validity)

    return model

# Create instances of the models
generator_cubism = build_generator()
discriminator_cubism = build_discriminator()

# Compile models with appropriate optimizer and loss
optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
binary_crossentropy_loss = BinaryCrossentropy()

generator_cubism.compile(optimizer=optimizer, loss=style_transfer_loss)
discriminator_cubism.compile(optimizer=optimizer, loss=binary_crossentropy_loss, metrics=['accuracy'])

In [35]:
gan_cubism = build_gan(generator_cubism, discriminator_cubism)
gan_cubism.compile(optimizer=optimizer, loss=style_transfer_loss)

ValueError: Exception encountered when calling layer "sequential_16" (type Sequential).

Input 0 of layer "dense_6" is incompatible with the layer: expected axis -1 of input shape to have value 524288, but received input with shape (None, 8388608)

Call arguments received by layer "sequential_16" (type Sequential):
  • inputs=tf.Tensor(shape=(None, 1024, 1024, 3), dtype=float32)
  • training=None
  • mask=None